In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import skimage.transform
import matplotlib.pyplot as plt

import itertools
from typing import Tuple

In [ ]:
tf.test.is_gpu_available()

## Load training data

Load data and create shifted versions.  
This is not redundant, even though we are using convolutional layers, as there are boundary effects (especially with the very small size).

In [ ]:
WIDTH = 5
HEIGHT = 5

with open("data.txt") as f:
    t = f.read()

xs = []
ys = []

for ex in t.split("\n\n"):
    lines = ex.split("\n")
    y  = int(lines[0])
    x = np.array([[0 if x == "." else 1 for x in line] for line in lines[1:]])
    
    ex_width = np.max(np.arange(1, WIDTH + 1) * x)
    ex_height = np.max((np.arange(1, HEIGHT + 1) * x.T).T)
    print(y, ex_width, ex_height, end="")
    for offset_x, offset_y in itertools.product(range(WIDTH - ex_width + 1),
                                                range(HEIGHT - ex_height + 1)):
        xs.append(np.roll(x, (offset_y, offset_x), axis=(0, 1)))
        ys.append(y)
        print(".", end="")
    print()

xs = np.array(xs).astype(float)[..., np.newaxis]
ys = np.array(ys)

\# of (shifted) examples per class

In [ ]:
len(ys)

In [ ]:
pd.Series(ys).value_counts()

In [ ]:
i = np.random.randint(0, len(ys))
print(ys[i])
print(xs[i,:,:,0].astype(int))

## Add augmentation & create tf dataset

In [ ]:
def random_flip(x: np.ndarray, xs_strings: np.ndarray) -> np.ndarray:
    """Randomly flips a bit in x, ensuring that the resulting pattern
       does not occurr in xs_strings
    """
    while True:
        x_new = x.copy()
        pos_y = np.random.randint(0, HEIGHT)
        pos_x = np.random.randint(0, WIDTH)
        x_new[pos_y, pos_x] = 1 - x_new[pos_y, pos_x]
        if x_new.tostring() not in xs_strings:
            return x_new

In [ ]:
xs_strings = {x.tostring() for x in xs}

In [ ]:
ys_onehot = tf.keras.utils.to_categorical(ys)

In [ ]:
BATCH_SIZE = 512

ds = tf.data.Dataset.zip((
    tf.data.Dataset.from_tensor_slices(xs),
    tf.data.Dataset.from_tensor_slices(ys_onehot)
)).shuffle(buffer_size=len(ys)).repeat()

def augment(x: np.ndarray) -> np.ndarray:
    """Flip up to 3 bits randomly"""
    if np.random.uniform() > 0.6:
        x = random_flip(x, xs_strings)
    if np.random.uniform() > 0.85:
        x = random_flip(x, xs_strings)
    if np.random.uniform() > 0.95:
        x = random_flip(x, xs_strings)
    
    return x
    

def map_fn(x: tf.Tensor, y: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = tf.numpy_function(func=augment , inp=[x], Tout=tf.float64)
    x.set_shape([HEIGHT, WIDTH, 1])
    return x, y

ds = ds.map(map_fn, num_parallel_calls=8)
ds = ds.batch(BATCH_SIZE)


## Define model

regularization actually improves accuracy on the non-augmented (flipped) samples and makes led activation patterns nicer.

In [ ]:
l2_reg = 0.004

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        4, 2,
        activation="relu", padding="valid",
        kernel_regularizer=tf.keras.regularizers.L1L2(l2=l2_reg),
        bias_regularizer=tf.keras.regularizers.L1L2(l2=l2_reg),
        input_shape=(WIDTH, HEIGHT, 1)),
    tf.keras.layers.Conv2D(8, 2, activation="relu",
        kernel_regularizer=tf.keras.regularizers.L1L2(l2=l2_reg),
        bias_regularizer=tf.keras.regularizers.L1L2(l2=l2_reg),),
    tf.keras.layers.Conv2D(16, 2, activation="relu",
        kernel_regularizer=tf.keras.regularizers.L1L2(l2=l2_reg),
        bias_regularizer=tf.keras.regularizers.L1L2(l2=l2_reg),),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax)
])

In [ ]:
model.count_params()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025),
    metrics=["accuracy"],
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.0))

## Train

In [ ]:
# training is fast, so let's just go crazy with the number of epochs
for i in range(100):
    model.fit(ds, epochs=100, verbose=0, steps_per_epoch=len(ys) // BATCH_SIZE)
    print(model.evaluate(xs, ys_onehot, verbose=0))

In [ ]:
model.optimizer.lr.assign(0.0005)
for i in range(40):
    model.fit(ds, epochs=100, verbose=0, steps_per_epoch=len(ys) // BATCH_SIZE)
    print(model.evaluate(xs, ys_onehot, verbose=0))

In [ ]:
model.optimizer.lr.assign(0.00005)
for i in range(40):
    model.fit(ds, epochs=100, verbose=0, steps_per_epoch=len(ys) // BATCH_SIZE)
    print(model.evaluate(xs, ys_onehot, verbose=0))

In [ ]:
model.evaluate(ds, steps=len(ys) // BATCH_SIZE)

In [ ]:
print(model.evaluate(xs, ys_onehot, verbose=0))

In [ ]:
preds = model.predict(xs)
pred_classes = np.argmax(preds, axis=1)
wrong = pred_classes != ys
display(xs[wrong][...,0])
display(preds[wrong])

In [ ]:
model.save("5x5_4-8-16_filters.savedmodel")

In [ ]:
model = tf.keras.models.load_model("5x5_4-8-16_filters.savedmodel/")

In [ ]:
model.summary()

In [ ]:
test_input = np.array([
    [0,0,0,0,0],
    [0,1,1,1,0],
    [0,1,1,1,0],
    [0,0,0,1,0],
    [0,1,1,1,0]
    ])[np.newaxis,..., np.newaxis].astype(np.float32)

In [ ]:
model(test_input)

## Save weights

In [ ]:
for i in range(3):
    base_name = f"weights/conv{i}_{{}}.npy"
    kernel, bias = model.layers[i].weights
    np.save(base_name.format("kernel"), kernel.numpy())
    np.save(base_name.format("bias"), bias.numpy())
    
kernel, bias = model.layers[4].weights
np.save("weights/dense_kernel.npy", kernel.numpy())
np.save("weights/dense_bias.npy", bias.numpy())

# Output model for inclusion in C code

## Get 99th percentile activation strengths for led brightness

In [ ]:
x = xs
for i in range(3):
    x = model.layers[i](x)
    perc_99 = np.percentile(x, 99)
    print(f"const float conv{i}_activation_99per = {np.format_float_positional(perc_99)};")

## Print weights

In [ ]:
def print_float_array(name: str, x: np.ndarray) -> None:
    assert(len(x.shape) == 1)
    print(f"const float {name}[] = {{")
    for i in range(int(np.ceil(len(x) / 5))):
        print("    " + ", ".join(np.format_float_scientific(f) for f in x[i*5: i*5 + 5]) + ",")
    print("};")

In [ ]:
for i in range(3):
    print_float_array(
        f"conv{i}_kernel_data",
        model.layers[i].weights[0].numpy().flatten()
    )
    print()
    print_float_array(
        f"conv{i}_bias_data",
        model.layers[i].weights[1].numpy().flatten()
    )
    print()
    
print_float_array(
    "dense_kernel_data",
    model.layers[4].weights[0].numpy().T.flatten()
)
print()
print_float_array(
    "dense_bias_data",
    model.layers[4].weights[1].numpy().flatten()
)